In [1]:
import os
import sys

#Import config file. Update config.py according to your environment
import config

import pandas as pd
import numpy as np

import requests, json

from tqdm import tqdm
import time
from datetime import datetime

In [7]:
start_year = 2022
end_year = 2025
current_year = datetime.now().year
year_list = list(range(start_year,end_year + 1,1))
citecount_list = (current_year-2) - np.array(year_list)
citecount_list[citecount_list < 0] = 0
citecount_list
 
fieldsOfStudy = 'Computer Science,Biology'#,Medicine,Physics,Geology,Psychology,Mathematics,Environmental Science,Agricultural and Food Sciences'
fields2return = 'title,citationCount,abstract,venue,authors,publicationDate,fieldsOfStudy'
additional_specs = 'influentialCitationCount,openAccessPdf,references'#,authors.affiliations,references.paperId,embedding.specter_v2'

## Downloading Semantics Scholar dataset

In [ ]:
def merge_dicts(dict_list):
    merged = {}
    for d in dict_list:
        for key, value in d.items():
            merged.setdefault(key, []).append(value)  # Collect values as lists
    return merged

data_source = 'semanticsscholar'
metadata_file_path = os.path.join(config.path_to_data, 'metadata')
text_file_path = os.path.join(config.path_to_data, 'text')
Foverwrite = True

for i, (year, minCitationCount) in enumerate(zip(year_list, citecount_list)):
    year_metadata_dir = os.path.join(metadata_file_path, str(year))
    year_text_dir = os.path.join(text_file_path, str(year))
    os.makedirs(year_metadata_dir, exist_ok=True)
    os.makedirs(year_text_dir, exist_ok=True)

    articles = None
    endpoint = f'https://api.semanticscholar.org/graph/v1/paper/search/bulk?fields={fields2return}&fieldsOfStudy={fieldsOfStudy}&minCitationCount={str(minCitationCount)}&year={str(year)}&openAccessPdf'
    dataset_list = None
    waitsec = 1
    while (dataset_list is None or 'total' not in dataset_list.keys()) and waitsec < 100:
        dataset_list = requests.get(endpoint).json()
        if 'total' not in dataset_list.keys():
            print(dataset_list)
        time.sleep(waitsec)
        waitsec += 1

    N = dataset_list['total']
    N_dw = 0
    batch_size = len(dataset_list['data'])
    token = 0
    old_len = 0
    with tqdm(range(N//batch_size + 1), desc=f'Year {year} / {len(year_list)} years ({N} papers on that year)', unit='batch') as pbar:
        for k in pbar:
            if token is not None:
                token_str = f'&token={token}' if k > 0 else ''
                
                dataset_list = None
                waitsec = 1
                while (dataset_list is None or 'total' not in dataset_list.keys() or (dataset_list['token'] is None and k < N//batch_size)) and waitsec < 100:
                    pbar.set_postfix({'status': f'loading batch, attempt #{waitsec}', 'token': token})
                    dataset_list = requests.get(endpoint + token_str).json()
                    if 'total' not in dataset_list.keys() or (dataset_list['token'] is None and k < N//batch_size):
                        print(dataset_list)
                    time.sleep(waitsec)
                    waitsec += 1

                token = dataset_list['token']
                pbar.set_postfix({'status': 'batch loaded', 'token': token})

                df = pd.DataFrame(dataset_list)
                N_dw += len(df)
                df = pd.concat([df.drop(columns=['data','token','total']), pd.json_normalize(df['data'])], axis=1)
                df = pd.concat([df.drop(columns=['authors']), pd.json_normalize(df['authors'].apply(merge_dicts))], axis=1)
                df = df.dropna(subset=['publicationDate','name','fieldsOfStudy']).drop_duplicates(subset='title')
                df = df[df['venue'].str.len()>3]
                df['authorId'] = df['authorId'].apply(lambda x: ','.join([name for name in x if name is not None]))
                df['authorName'] = df['name'].apply(lambda x: ','.join(x))
                df = df.drop(columns='name')
                df['fieldsOfStudy'] = df['fieldsOfStudy'].apply(lambda x: ','.join([name for name in x if name is not None]))
                df['publicationYear'] = year

                if additional_specs:
                    B = len(df)
                    minibatch_size = 500
                    df_specs = pd.DataFrame()
                    for k in range(B//minibatch_size + 1):
                        r = None
                        waitsec = 1
                        while (r is None or type(r) is not list) and waitsec < 100:
                            try:
                                r = requests.post('https://api.semanticscholar.org/graph/v1/paper/batch',
                                                params={'fields': additional_specs},
                                                json={"ids": df.paperId[k*minibatch_size:min(B,(k+1)*minibatch_size)].to_list()}
                                                ).json()
                            except requests.exceptions.JSONDecodeError:
                                print("Invalid JSON response received. Trying again")
                            time.sleep(waitsec)
                            waitsec += 1

                        r = [doc for doc in r if doc is not None]
                        df_temp = pd.DataFrame(r)
                        if any(spec in df_temp.columns for spec in additional_specs.split(',')):
                            df_temp['openAccessPdf'] = df_temp['openAccessPdf'].apply(lambda x: x['url'] if x is not None else x)
                            df_temp['referenceIds'] = df_temp['references'].apply(lambda x: ';'.join([ref['paperId'] for ref in x if ref['paperId'] is not None]))
                            df_temp['referenceTitles'] = df_temp['references'].apply(lambda x: ';'.join([ref['title'].replace(';', ',') for ref in x if ref if ref['title'] is not None]))
                            df_temp = df_temp.drop(columns=['references'])
                            df_temp['influentialCitationCount'] = df_temp['influentialCitationCount'].astype(float).fillna(0)
                            df_specs = pd.concat([df_specs, df_temp], axis=0)
                    
                    df = pd.merge(left=df, right=df_specs, how='left', left_on='paperId', right_on='paperId')
                    df['influentialCitationCount'] = df['influentialCitationCount'].fillna(0)
                    df['openAccessPdf'] = df['openAccessPdf'].fillna('None')

                    df = df[['paperId', 'title', 'abstract', 'openAccessPdf', 'fieldsOfStudy', 'venue', 'authorName', 'authorId', 'citationCount', 'influentialCitationCount', 'publicationDate', 'publicationYear', 'referenceIds', 'referenceTitles']]
                    pbar.set_postfix({'status': 'additional specs loaded', 'token': token})

                else:
                    df = df[['paperId', 'title', 'abstract', 'fieldsOfStudy', 'venue', 'authorName', 'authorId', 'citationCount', 'publicationDate', 'publicationYear']]

                if articles is None:
                    articles = df
                else:
                    articles = pd.concat([articles, df], axis=0)
                    articles = articles.drop_duplicates(subset='paperId')

                new_articles = len(articles) - old_len
                old_len = len(articles)
                pbar.set_postfix({'status': f"{new_articles} articles added", 'token': token})
            else:
                print(f"end of dataset at batch {k} / {N//batch_size} token: {token}")
                if k < N//batch_size - 1:
                    pbar.close()
                    raise ValueError("Returned token is None before end of dataset")
            
    articles = articles.reset_index(drop=True)
    str_col = articles.columns[articles.dtypes==object]
    for col in str_col:
        articles[col] = articles[col].astype(str).str.strip().str.encode('utf-8', 'ignore').str.decode('utf-8')

    print(f'Total number of articles for {year}: {len(articles)} ({N_dw} downloaded)')

    year_metadata_file_path = os.path.join(year_metadata_dir, f'{data_source}_metadata_{year}.parquet')
    year_text_file_path = os.path.join(year_text_dir, f'{data_source}_text_{year}.parquet')
    
    metadata = articles.drop(columns=['title', 'abstract'])
    metadata.to_parquet(year_metadata_file_path, engine="pyarrow", compression="snappy", index=True)

    text = articles[['paperId', 'title', 'abstract', 'referenceTitles', 'openAccessPdf']]
    text.to_parquet(year_text_file_path, engine="pyarrow", compression="snappy", index=True)

Year 2022 / 4 years (428586 papers on that year): 100%|██████████| 429/429 [3:57:07<00:00, 33.16s/batch, status=426 articles added, token=None]                                                                                                                                  


Total number of articles for 2022: 311076 (428598 downloaded)


Year 2023 / 4 years (578172 papers on that year): 100%|██████████| 579/579 [4:04:46<00:00, 25.37s/batch, status=104 articles added, token=None]                                                                                                                                  


Total number of articles for 2023: 345306 (578170 downloaded)


Year 2024 / 4 years (307328 papers on that year): 100%|██████████| 308/308 [1:59:00<00:00, 23.18s/batch, status=196 articles added, token=None]                                                                                                                                


Total number of articles for 2024: 174994 (307317 downloaded)


Year 2025 / 4 years (3414 papers on that year): 100%|██████████| 4/4 [01:31<00:00, 22.94s/batch, status=268 articles added, token=None]                                                                                                                                


Total number of articles for 2025: 2239 (3414 downloaded)


In [76]:
pdf_text = extract_text(os.path.join(config.path_to_data, 'pdf', '002dea70291555c99aa2d88b87db7e8d3ccad66a.pdf'))

def process_pdf(pdf_text):
    
    # Find sections. We’ll guess some common next headings for each section:
    abstract_text = find_section(pdf_text, "abstract", 
                                 ["\n\n"], max_chars=3000)
    introduction_text = find_section(pdf_text, "introduction", 
                                     ["methods", "methodology", "materials and methods", "results", "discussion"], max_chars=5000)
    discussion_text = find_section(pdf_text, "discussion", 
                                   ["conclusion", "references", "acknowledgements"], max_chars=5000)
    
    all_text = find_section(pdf_text, " ", 
                                   ["conclusion", "references", "acknowledgements"], max_chars=5000)
    
    return abstract_text, introduction_text, discussion_text, all_text

abstract_text, introduction_text, discussion_text, all_text = process_pdf(pdf_text)

In [124]:
censored_List = ['laboratory', 'received', 'accepted', 'corresponding']
pdf_text = all_text.split('\n') #pdf_text.split('.')
pdf_text = [txt for txt in pdf_text if len(txt.split()) > 3 and not any(censored.lower() in txt.lower().split() for censored in censored_List)]

# k = 0
# while k < len(pdf_text)-1:
#     if len(pdf_text[k].split()) < 150:
#         pdf_text[k] = pdf_text[k] + pdf_text[k+1]
#         pdf_text.pop(k+1)
#     else:
#         k +=1
[txt for txt in pdf_text]

['MOLECULAR AND CELLULAR BIOLOGY, May 2005, p. 4166–4175',
 'Copyright © 2005, American Society for Microbiology. All Rights Reserved.',
 'Vol. 25, No. 10',
 'Neuronal Leucine-Rich Repeat Protein 4 Functions in',
 'Takayoshi Bando,1 Keisuke Sekine,1 Shizuka Kobayashi,3 Ayako M. Watabe,3 Armin Rump,1',
 'Minoru Tanaka,1 Yoshikuni Suda,1 Shigeaki Kato,2 Yoshihiro Morikawa,4',
 'Toshiya Manabe,3 and Atsushi Miyajima1*',
 'Cellular Biosciences, University of Tokyo, Yayoi, Bunkyo-ku, Tokyo 113-0032, Japan; Division of Neuronal',
 'Network, Department of Basic Medical Sciences, Institute of Medical Science, University of Tokyo, 4-6-1',
 'Shirokanedai, Minato-ku, Tokyo 108-8639, Japan3; and Department of Anatomy and Neurobiology,',
 'Wakayama Medical University, 811-1 Kimidera, Wakayama 641-8509, Japan4',
 'Neuronal leucine-rich repeat proteins (NLRRs) are type I transmembrane proteins and expressed in',
 'neuronal tissues, but their function remains unknown. Here, we describe the identiﬁcati

In [126]:
import torch
from transformers import AutoTokenizer, AutoModel
from adapters import AutoAdapterModel

model_name = 'sentence-transformers/all-mpnet-base-v1' #'allenai/specter2'#
batch_size = 128
max_length = 512
Nsamples = len(pdf_text)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if 'sentence-transformer' in model_name:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
elif 'specter2' in model_name:
    tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')
    model = AutoAdapterModel.from_pretrained('allenai/specter2_base')
    model.load_adapter(model_name, source="hf", set_active=True)

model.to(device);
for param in model.parameters():
    param.requires_grad = False

if 'sentence-transformer' in model_name:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model_query = AutoModel.from_pretrained(model_name)
elif 'specter2' in model_name:
    tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')
    model_query = AutoAdapterModel.from_pretrained('allenai/specter2_base')
    model_query.load_adapter("allenai/specter2_adhoc_query", source="hf", set_active=True)

model_query.to(device);
for param in model_query.parameters():
    param.requires_grad = False

In [127]:
embeddings = []
model.eval()
with torch.no_grad():
    with tqdm(range(Nsamples//batch_size + 1), desc=f'total of {Nsamples} papers', unit='batch') as pbar:
        for i in pbar:
            batch_text = pdf_text[i*batch_size:(i + 1)*batch_size]
            if batch_text:
                batch_tokens = tokenizer(batch_text, padding=True, truncation=True, return_tensors="pt", return_token_type_ids=False, max_length=max_length)
                for key in batch_tokens.keys():
                    batch_tokens[key] = batch_tokens[key].to(device)
                
                output = model(**batch_tokens)
                # first token in the batch as the embedding
                if 'pooler_output' in output.keys():
                    embeddings_batch = output.pooler_output.cpu().numpy().astype(np.float32)
                else:
                    embeddings_batch = output.last_hidden_state[:, 0, :].cpu().numpy().astype(np.float32)
                    
                embeddings.append(embeddings_batch)

total of 524 papers: 100%|██████████| 5/5 [00:00<00:00,  5.02batch/s]


In [128]:
embeddings = np.concat(embeddings, axis=0)

In [129]:
#normalize embeddings
embeddings_norm = embeddings / np.linalg.norm(embeddings,axis=-1, keepdims=True)

In [138]:
import numpy as np
import umap
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# Step 1: Reduce Dimensionality with UMAP (or PCA)
umap_reducer = umap.UMAP(n_components=2, metric='cosine', random_state=42)
reduced_embeddings = umap_reducer.fit_transform(embeddings_norm)

# Step 2: Apply DBSCAN Clustering
dbscan = DBSCAN(eps=0.5, min_samples=25, metric='euclidean')  # Tune eps and min_samples
labels = dbscan.fit_predict(reduced_embeddings)

# Step 3: Identify and Remove Outliers (DBSCAN assigns -1 to outliers)
filtered_embeddings = embeddings_norm[labels != -1]
filtered_labels = labels[labels != -1]
filtered_text = np.array(pdf_text)[labels != -1]

/home/jul/anaconda3/envs/recoRAG/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jul/anaconda3/envs/recoRAG/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [139]:
print(filtered_text)

['Neuronal Leucine-Rich Repeat Protein 4 Functions in'
 'Neuronal leucine-rich repeat proteins (NLRRs) are type I transmembrane proteins and expressed in'
 'tion of a new member of the NLRR family, NLRR4, and its potential role in long-lasting memory. We generated'
 'ported, such as N-methyl-D-aspartate receptor, calcium-calm-'
 'In this study, we describe a novel type I transmembrane'
 'protein termed NLRR4 which exhibits similarity to neuronal'
 'leucine-rich repeat proteins NLRR1 to NLRR3. NLRR1 and'
 'NLRR2 are expressed in the developing nervous system (4, 14,'
 '28, 29). NLRR3 is induced by brain injury and regulated by'
 'However, their functions remain to be studied. To uncover the'
 'function of NLRR4 in vivo, we generated NLRR4-deﬁcient'
 'dase gene by homologous'
 'normal. These data indicate that NLRR4 is important for'
 'Expression cloning of NLRR4 cDNA. Expression cloning of a cDNA encoding'
 'FIG. 1. Structural properties of NLRR4. (a) The deduced amino acid sequences of

In [117]:
cluster_id=10
for k, txt in enumerate(filtered_text):
    if filtered_labels[k]==cluster_id:
        print(k, sum(labels==cluster_id))
        print(txt)
        print()
        print()


In [36]:
print(' '.join(pdf_text))

MOLECULAR AND CELLULAR BIOLOGY, May 2005, p 2005
Copyright © 2005, American Society for Microbiology  10

Neuronal Leucine-Rich Repeat Protein 4 Functions in
Hippocampus-Dependent Long-Lasting Memory
Takayoshi Bando,1 Keisuke Sekine,1 Shizuka Kobayashi,3 Ayako M  Watabe,3 Armin Rump,1
Minoru Tanaka,1 Yoshikuni Suda,1 Shigeaki Kato,2 Yoshihiro Morikawa,4
Toshiya Manabe,3 and Atsushi Miyajima1*
Laboratory of Cell Growth and Differentiation1 and Laboratory of Neuclear Signaling,2 Institute of Molecular and
Cellular Biosciences, University of Tokyo, Yayoi, Bunkyo-ku, Tokyo 113-0032, Japan; Division of Neuronal
Network, Department of Basic Medical Sciences, Institute of Medical Science, University of Tokyo, 4-6-1
Shirokanedai, Minato-ku, Tokyo 108-8639, Japan3; and Department of Anatomy and Neurobiology,
Wakayama Medical University, 811-1 Kimidera, Wakayama 641-8509, Japan4

Received 11 November 2004/Accepted 13 February 2005

Neuronal leucine-rich repeat proteins (NLRRs) are type I transme

In [50]:
query = "Neuronal leucine-rich repeat proteins (NLRRs) are type I transmembrane proteins and expressed in neuronal tissues"
query_input = tokenizer([query], padding=True, truncation=True,
                              return_tensors="pt", return_token_type_ids=False, max_length=max_length)

for key in query_input.keys():
    query_input[key] = query_input[key].to(device)
with torch.no_grad():
    output = model_query(**query_input)
    if 'pooler_output' in output.keys():
        embs = output.pooler_output.cpu().numpy()
    else:
        embs = output.last_hidden_state[:, 0, :].cpu().numpy()

#normalize embedding
embs_norm = np.median(embeddings, axis=0, keepdims=True)
embs_norm = embs / np.linalg.norm(embs,axis=-1, keepdims=True)

In [51]:
#Compute cos similarity and find topK
K = len(pdf_text)
similarity_matrix = np.einsum('bc,kc->bk', embs_norm, embeddings_norm)
topK_indices = np.argsort(-similarity_matrix, axis=-1)[:,:K]
topK_values = np.take_along_axis(similarity_matrix, topK_indices, axis=-1)

topK_indices = topK_indices[0]
topK_values = topK_values[0]

In [52]:
K

331

In [53]:
for k in topK_indices:
    if len(pdf_text[k].split(' '))>15:
        print(k)
        print(pdf_text[k])

22


In this study, we describe a novel type I transmembrane
protein termed NLRR4 which exhibits similarity to neuronal
leucine-rich repeat proteins NLRR1 to NLRR3
102
 Due
to the similarity to the previously identiﬁed neuronal leucine
rich-repeat proteins NLRR1 to NLRR3 (4, 14, 28, 29), we
named B61 NLRR4
155
 In the adult brain, NLRR4 was strongly
expressed in the hippocampus and weakly in the cerebellum
(Fig
311
 Since NLRR4 is highly expressed in the
hippocampus, it is possible that NLRR4 is involved in learning
and/or memory
324
 Axon of
CA1 pyramidal cells mainly extend to the layer V in entorhinal

cortex and NLRR4 is expressed in CA1, CA3, and dentate
gyrus in the hippocampus and in layers V and VI in the cortex
2
 10

Neuronal Leucine-Rich Repeat Protein 4 Functions in
Hippocampus-Dependent Long-Lasting Memory
Takayoshi Bando,1 Keisuke Sekine,1 Shizuka Kobayashi,3 Ayako M
4
 Here, we describe the identiﬁcation and characteriza-
tion of a new member of the NLRR family, NLRR4, a

In [87]:
filtered_metadata = pd.read_parquet(file_path + '_metadata.parquet', engine="pyarrow", filters=[("citationCount", ">", 100)])
filtered_metadata

,paperId,openAccessPdf,fieldsOfStudy,venue,authorName,authorId,citationCount,influentialCitationCount,publicationDate,publicationYear
1,000372290caae5482dcdd7954feb895ccbab921d,http://www.merl.com/publications/docs/TR2005-0...,"Computer Science,Mathematics",IEEE Transactions on Communications,"Juntan Zhang,M. Fossorier","3193350,48891339",319,26.0,2005-03-07,2005
5,0004b93367db8768d29c7e21bf410d55a30f5205,http://dl.acm.org/ft_gateway.cfm?id=1047480&ty...,Computer Science,Technical Symposium on Computer Science Education,"Susan Bergin,R. Reilly","145985158,2688685",287,26.0,2005-02-23,2005
6,0004e56a361394b92b9ac1d15a524ef7ecfe2767,https://www.ahajournals.org/doi/pdf/10.1161/01...,"Medicine,Biology","Arteriosclerosis, Thrombosis and Vascular Biology","E. Egorina,M. Sovershaev,G. Bjørkøy,F. Gruber,...","8887769,4962786,11414707,50148632,33502527,139...",148,2.0,2005-07-01,2005
7,000646db4a4a269f9261f5b285ba1d07ef006809,https://europepmc.org/articles/pmc549252?pdf=r...,"Biology,Medicine",Antimicrobial Agents and Chemotherapy,"T. Levin,B. Suh,P. Axelrod,A. Truant,T. Fekete","24218070,1941629,144104608,4961128,49307125",179,10.0,2005-03-01,2005
23,000fd65de92d766ada2dab424f7f66e526e2f063,https://infoscience.epfl.ch/record/87259/files...,Computer Science,IEEE transactions on circuits and systems for ...,"A. Cavallaro,Olivier Steiger,T. Ebrahimi","145440152,2359024,1681498",120,7.0,2005-10-01,2005
...,...,...,...,...,...,...,...,...,...,...
6964,07eb0e2775f4c22f59c7049474fe631921aaea1a,https://www.jneurosci.org/content/jneuro/26/1/...,"Psychology,Medicine",Journal of Neuroscience,"Julien Voisin,A. Bidet-Caulet,O. Bertrand,P. F...","144092326,1401757187,35179356,3195672",153,5.0,2006-01-04,2006
6966,07ec944496ff49fa128b004cf3cdeb302ae2c243,https://europepmc.org/articles/pmc3565218?pdf=...,"Biology,Medicine",Developmental Biology,"C. A. Whittaker,K. Bergeron,J. Whittle,B. Bran...","49692462,66505346,4519819,5628946,34842592,355...",179,13.0,2006-12-01,2006
6969,07ef49f1a7de858badbab788dc93f027fdf91980,https://academic.oup.com/endo/article-pdf/147/...,"Biology,Medicine",Endocrinology,"Wan Huang,N. Dedousis,Archana Bandi,G. Lopasch...","47504581,3650543,40587456,4829939,1397132755",121,3.0,2006-03-01,2006
6975,07f45af2cd5df5ae6795fe1b6c0fabef0d543b63,https://europepmc.org/articles/pmc1797767?pdf=...,"Biology,Medicine",Antimicrobial Agents and Chemotherapy,"R. Shandil,R. Jayaram,P. Kaur,S. Gaonkar,B. L....","152540199,13981673,2192681880,34197093,4017113...",212,16.0,2006-12-04,2006


In [93]:
filtered_ids = filtered_metadata.index.tolist()
filtered_text = pd.read_parquet(file_path + '_text.parquet', engine="pyarrow", filters=[("__index_level_0__", "in", filtered_ids)])
filtered_text

,paperId,title,abstract,openAccessPdf
1,000372290caae5482dcdd7954feb895ccbab921d,Shuffled iterative decoding,Shuffled versions of iterative decoding of low...,http://www.merl.com/publications/docs/TR2005-0...
5,0004b93367db8768d29c7e21bf410d55a30f5205,Programming: factors that influence success,"This paper documents a study, carried out in t...",http://dl.acm.org/ft_gateway.cfm?id=1047480&ty...
6,0004e56a361394b92b9ac1d15a524ef7ecfe2767,Intracellular and Surface Distribution of Mono...,Objective—The high and low responder phenomeno...,https://www.ahajournals.org/doi/pdf/10.1161/01...
7,000646db4a4a269f9261f5b285ba1d07ef006809,Potential Clindamycin Resistance in Clindamyci...,ABSTRACT The erm gene product confers clindamy...,https://europepmc.org/articles/pmc549252?pdf=r...
23,000fd65de92d766ada2dab424f7f66e526e2f063,Semantic video analysis for adaptive content d...,We present an encoding framework which exploit...,https://infoscience.epfl.ch/record/87259/files...
...,...,...,...,...
6964,07eb0e2775f4c22f59c7049474fe631921aaea1a,Listening in Silence Activates Auditory Areas:...,Directing attention to some acoustic features ...,https://www.jneurosci.org/content/jneuro/26/1/...
6966,07ec944496ff49fa128b004cf3cdeb302ae2c243,The echinoderm adhesome.,None,https://europepmc.org/articles/pmc3565218?pdf=...
6969,07ef49f1a7de858badbab788dc93f027fdf91980,Liver triglyceride secretion and lipid oxidati...,Leptin has potent lipid-lowering effects in pe...,https://academic.oup.com/endo/article-pdf/147/...
6975,07f45af2cd5df5ae6795fe1b6c0fabef0d543b63,"Moxifloxacin, Ofloxacin, Sparfloxacin, and Cip...",ABSTRACT Members of the fluoroquinolone class ...,https://europepmc.org/articles/pmc1797767?pdf=...


In [35]:
print(max_itemsize)
str_col = articles.columns[articles.dtypes==object]

max_len = articles['authorName'].apply(lambda x: len(x.encode('utf-8')[:4785].decode('utf-8')) if isinstance(x, str) else 0)
for i in range(len(articles)):
    if max_len.iloc[i] >=4790:
        print(articles['authorName'].iloc[i])
    # max_bytes = articles[col].apply(lambda x: len(x.encode('utf-8')) if isinstance(x, str) else 0).max()
    # print(articles[col].astype(str).str.strip().str.encode('utf-8', 'ignore').str.decode('utf-8').str.len().max(), max_bytes)

{'title': np.int64(450), 'abstract': np.int64(15000), 'openAccessPdf': np.int64(639), 'fieldsOfStudy': np.int64(91), 'venue': np.int64(301), 'authorName': np.int64(4785), 'authorId': np.int64(3916), 'publicationDate': np.int64(15)}


In [ ]:
fields2return = 'influentialCitationCount,authors.affiliations,references.paperId,embedding.specter_v2'
r = requests.post('https://api.semanticscholar.org/graph/v1/paper/batch',
                            params={'fields': fields2return},
                            json={"ids": articles.paperId[:500].to_list()}
                        ).json()

In [ ]:
r = requests.post(
    'https://api.semanticscholar.org/graph/v1/paper/batch',
    params={'fields': 'title,abstract,venue,publicationVenue,year,citationCount,influentialCitationCount,openAccessPdf,authors.name,authors.affiliations,authors.paperCount,authors.citationCount,authors.hIndex,embedding.specter_v2,references.paperId'},
    json={"ids": ['10.1016/j.cub.2020.07.006']}
).json()

In [50]:
data_path = os.path.join(config.path_to_data, 'articles_semnatics.h5')
with pd.HDFStore(data_path, mode="r") as store:
    data = store.select("articles/main")#, where="publicationYear == 2022")
display(data)


,title,abstract,openAccessPdf,fieldsOfStudy,venue,authorName,authorId,citationCount,influentialCitationCount,publicationDate,publicationYear
paperId,,,,,,,,,,,
0001d5fbff6f7c763a78dd5d141292416dcfae59,Disruption of Fibroblast Growth Factor Signal ...,Purpose: Synovial sarcoma is a soft tissue sar...,https://aacrjournals.org/clincancerres/article...,"Biology,Medicine",Clinical Cancer Research,"T. Ishibe,T. Nakayama,T. Okamoto,T. Aoyama,K. ...","14839966,10239302,2000448727,3201672,7147569,2...",88,6.0,2005-04-01,2005
000372290caae5482dcdd7954feb895ccbab921d,Shuffled iterative decoding,Shuffled versions of iterative decoding of low...,http://www.merl.com/publications/docs/TR2005-0...,"Computer Science,Mathematics",IEEE Transactions on Communications,"Juntan Zhang,M. Fossorier","3193350,48891339",319,26.0,2005-03-07,2005
000376ec6f6ade6261fc12df9eddd25a88c1e9ca,Diphasic effects of Astragalus membranaceus BU...,This study was designed to investigate the eff...,https://www.jstage.jst.go.jp/article/bpb/28/8/...,Medicine,Biological and Pharmaceutical Bulletin,"Bi-qi Zhang,Shen-Jiang Hu,Lihong Qiu,Q. Shan,J...","11809650,6617669,123635715,11305673,2152146096...",24,2.0,2005-08-01,2005
0003a5747ee3676be1b6d2c72c13a04e41811274,Critical roles of CD146 in zebrafish vascular ...,"In this report, we use zebrafish as a model sy...",https://onlinelibrary.wiley.com/doi/pdfdirect/...,"Biology,Medicine",Developmental Dynamics,"Barden Chan,S. Sinha,Dan Cho,R. Ramchandran,V....","35690923,2114125286,2113252574,2134773,3658233",53,2.0,2005-01-01,2005
00043e28cffe744909bf69cde70e6dac0bce0a10,Differential Contribution of Troponin I Phosph...,Cardiac troponin I is a phosphorylation target...,http://www.jbc.org/article/S0021925820589394/pdf,"Chemistry,Medicine",Journal of Biological Chemistry,"M. Westfall,A. M. Lee,D. Robinson","5025983,49249911,48706510",70,3.0,2005-12-16,2005
...,...,...,...,...,...,...,...,...,...,...,...
ff32bd5f65db2b468501ee31d43600ef7140c90a,Escherichia coli O157 outbreak associated with...,SUMMARY A family cluster of three cases of Esc...,https://www.cambridge.org/core/services/aop-ca...,"Biology,Medicine",Epidemiology and Infection,"E. Espié,V. Vaillant,P. Mariani‐Kurkdjian,F. G...","32722639,144570095,1395683780,6424901,14229538...",129,2.0,2005-07-22,2005
ff342110ddcce74ca03a8f133a3f3ebd94ae5791,Induction of fibroblast growth factor-9 and in...,Motorcycle exhaust particulates (MEP) contain ...,https://academic.oup.com/toxsci/article-pdf/87...,"Biology,Medicine",Toxicological Sciences,"T. Ueng,Chia-Chi Hung,M. Kuo,P. Chan,Shih-Hsiu...","3850380,2064328719,49201256,87085105,2191091,2...",29,2.0,2005-10-01,2005
ff344be182b082370516662b41ace844028fde1f,A role for the CPF 3'-end processing machinery...,The prevailing view of the RNA polymerase II (...,http://genesdev.cshlp.org/content/19/24/2969.f...,"Medicine,Biology",Genes & Development,"A. Ansari,M. Hampsey","38324090,2899685",272,11.0,2005-12-15,2005


In [51]:
data['influentialCitationCount'].dtypes

dtype('float64')